# SQL 작성

In [1]:
import sys
import os
import pandas as pd
import cv2
import xml.etree.ElementTree as ET
sys.path.append('../../src')
import DISData as DD

## SQL 연결

In [3]:
doUT = DD.SQL()

host:  192.168.0.201
user:  diadmin
password:  ········


SQL connection done


## query 작성

In [4]:
query1 = (f'''CREATE TABLE IF NOT EXISTS DataVoucher2023_seatbelt_info(
            id INT PRIMARY KEY AUTO_INCREMENT,
            img_name VARCHAR(255) NOT NULL,
            img_dir VARCHAR(1000) NOT NULL,
            img_format VARCHAR(10) NOT NULL,
            img_width INT NOT NULL,
            img_height INT NOT NULL,
            color_space VARCHAR(10) NOT NULL,
            label_name VARCHAR(255) NOT NULL,
            label_dir VARCHAR(1000) NOT NULL,
            label_format VARCHAR(10) NOT NULL,
            label VARCHAR(20) NOT NULL,
            box_xtl INT,
            box_ytl INT,
            box_xbr INT,
            box_ybr INT,
            box_rate INT);''')

In [5]:
query2 = (f'''CREATE TABLE IF NOT EXISTS DataVoucher2022_landmark_info(
            id INT PRIMARY KEY AUTO_INCREMENT,
            img_name VARCHAR(255) NOT NULL,
            img_dir VARCHAR(1000) NOT NULL,
            img_format VARCHAR(10) NOT NULL,
            img_width INT NOT NULL,
            img_height INT NOT NULL,
            color_space VARCHAR(10) NOT NULL,
            label_name VARCHAR(255) NOT NULL,
            label_dir VARCHAR(1000) NOT NULL,
            label_format VARCHAR(10) NOT NULL,
            label VARCHAR(20) NOT NULL,
            landmark VARCHAR(1000));''')

In [6]:
query1_1 = (f'''INSERT INTO DataVoucher2022_box_info 
        (img_name, img_dir, img_format, img_width, img_height, color_space,
        label_name, label_dir, label_format,
        label, box_xtl, box_ytl, box_xbr, box_ybr, box_rate)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')
query2_1 = (f'''INSERT INTO DataVoucher2022_landmark_info 
        (img_name, img_dir, img_format, img_width, img_height, color_space,
        label_name, label_dir, label_format,
        label, landmark)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')

## query 실행

In [7]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [8]:
label_path = 'Z:\\DataVoucher2022\\data\\IR\\labeldata'
label_lists = os.listdir(label_path)

value_list1 = []
value_list2 = []
n = 1
total_cnt = len(label_lists)

for cnt, label_list in enumerate(label_lists):
    
    label_files = os.listdir(os.path.join(label_path,label_list))
    for label_file in label_files:
        label = os.path.join(label_path,label_list,label_file)
    
        label_dir = '/'.join(os.path.dirname(label).split('\\')[-6:]+[os.path.basename(label)])
        label_name, label_format = os.path.splitext(os.path.basename(label))
        label_format = label_format.replace('.','')
        color_space = 'IR'
        # print(color_space)
        tree = ET.parse(label)
        root = tree.getroot()
        img_file = root.attrib['name']
        search_file = '\\'.join(os.path.dirname(label_dir).split('\\')+[img_file]).replace('label','raw')
        img_dir = '/'.join(os.path.dirname(search_file).split('\\')[-6:]+[img_file])
        img_name, img_format = os.path.splitext(img_file)
        img_format = img_format.replace('.','')
        img_width, img_height = int(root.attrib['width']), int(root.attrib['height'])
        img_area = img_width * img_height
        for i in root:
            tags = i.tag

            if tags == 'box':
                label = i.attrib['label']
                box_xtl, box_ytl, box_xbr, box_ybr = int(float(i.attrib['xtl'])), int(float(i.attrib['ytl'])), int(float(i.attrib['xbr'])), int(float(i.attrib['ybr']))
                box_area = abs(box_xtl - box_xbr) * abs(box_ytl - box_ybr)
                box_rate = round(box_area / img_area * 100,2) 
                img_dir = img_dir.replace('Z:','DataBase')
                label_dir = label_dir.replace('Z:','DataBase')
                value = (img_name, img_dir, img_format, img_width, img_height, color_space, label_name, label_dir, label_format, label, box_xtl, box_ytl, box_xbr, box_ybr, box_rate)
                value_list1.append(value)
                if len(value_list1) >= 500:
                    print(f'bbox {cnt} / {total_cnt}')
                    doUT.insert_dataset_values(query1_1, value_list1)
                    value_list1 = []
            else :
                label = i.tag
                landmark = ",".join(map(str, [(int(float(x)), int(float(y))) for x, y, z in [point.split(',') for point in i.attrib['points'].split(';')]]))
                img_dir = img_dir.replace('Z:','DataBase')
                label_dir = label_dir.replace('Z:','DataBase')
                value = (img_name, img_dir, img_format, img_width, img_height, color_space, label_name, label_dir, label_format, label, landmark)
                value_list2.append(value)
                if len(value_list2) >= 500:
                    # print(value_list1[0])
                    print(f'landmark {cnt} / {total_cnt}')
                    doUT.insert_dataset_values(query2_1, value_list2)
                    value_list2 = []
    
doUT.insert_dataset_values(query1_1, value_list1)
doUT.insert_dataset_values(query2_1, value_list2)
print('done')

landmark 2 / 197
insert 500 done
bbox 4 / 197
insert 500 done
landmark 5 / 197
insert 500 done
landmark 8 / 197
insert 500 done
bbox 9 / 197
insert 500 done
landmark 10 / 197
insert 500 done
landmark 13 / 197
insert 500 done
bbox 14 / 197
insert 500 done
landmark 16 / 197
insert 500 done
bbox 17 / 197
insert 500 done
landmark 18 / 197
insert 500 done
landmark 22 / 197
insert 500 done
landmark 25 / 197
insert 500 done
bbox 28 / 197
insert 500 done
landmark 28 / 197
insert 500 done
landmark 32 / 197
insert 500 done
landmark 36 / 197
insert 500 done
landmark 40 / 197
insert 500 done
landmark 43 / 197
insert 500 done
landmark 46 / 197
insert 500 done
landmark 49 / 197
insert 500 done
landmark 51 / 197
insert 500 done
landmark 54 / 197
insert 500 done
landmark 57 / 197
insert 500 done
bbox 60 / 197
insert 500 done
landmark 60 / 197
insert 500 done
landmark 62 / 197
insert 500 done
bbox 65 / 197
insert 500 done
landmark 65 / 197
insert 500 done
landmark 68 / 197
insert 500 done
bbox 70 / 197